In [ ]:
#Date: 9/26/2022
#Author: Talia Kurtz

#What is the purpose of this code?
#This code further drops unwanted variables from the ARM reduced winter files for each year and then NaNs data that was denoted in the quality control report as being incorrect or suspect. Once the
#incorrect and or suspect data is replaced as a NaN then the QC variables are run through each vairable and any value that deviates from 0 is then NaNed out. The individual variables are saved as seperate
#files to be globbed together into one large file.

In [1]:
#Imports
from datetime import datetime

import cfgrib
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
import scipy.ndimage as ndimage
import pandas as pd
import itertools

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#Open the ARM data file that I want 
ARM_PATH = '/Users/research/thesis_code/ARM_data_winter_months/'
dg = xr.open_dataset(ARM_PATH + 'ARM_2005_reduced.nc')

In [2]:
dx = dg.drop(['qc_logger_volt', 'pws_mean_vis_10min','qc_pws_mean_vis_10min', 'pws_pw_code_15min', 'qc_pws_pw_code_15min','pws_pw_code_1hr', 'qc_pws_pw_code_1hr'])
#dx = dg.drop(['qc_logger_volt', 'pwd_mean_vis_10min','qc_pwd_mean_vis_10min', 'pwd_pw_code_15min', 'qc_pwd_pw_code_15min','pwd_pw_code_1hr', 'qc_pwd_pw_code_1hr'])
dx

<xarray.Dataset>
Dimensions:                       (time: 5088)
Coordinates:
  * time                          (time) datetime64[ns] 2005-01-01 ... 2005-1...
Data variables: (12/28)
    qc_time                       (time) float64 ...
    atmos_pressure                (time) float32 ...
    qc_atmos_pressure             (time) float64 ...
    temp_mean                     (time) float32 ...
    qc_temp_mean                  (time) float64 ...
    temp_std                      (time) float32 ...
    ...                            ...
    qc_pws_precip_rate_mean_1min  (time) float64 ...
    pws_cumul_snow                (time) float32 ...
    qc_pws_cumul_snow             (time) float64 ...
    lat                           (time) float32 ...
    lon                           (time) float32 ...
    alt                           (time) float32 ...

In [3]:
start =datetime.strptime("2005-03-13 21", "%Y-%m-%d %H")
end = datetime.strptime("2005-03-15 00", "%Y-%m-%d %H")
date_generated = pd.date_range(start, end, freq='1H')
list_1 = date_generated.strftime("%Y-%m-%d %H")
print(list_1)

Index(['2005-03-13 21', '2005-03-13 22', '2005-03-13 23', '2005-03-14 00',
       '2005-03-14 01', '2005-03-14 02', '2005-03-14 03', '2005-03-14 04',
       '2005-03-14 05', '2005-03-14 06', '2005-03-14 07', '2005-03-14 08',
       '2005-03-14 09', '2005-03-14 10', '2005-03-14 11', '2005-03-14 12',
       '2005-03-14 13', '2005-03-14 14', '2005-03-14 15', '2005-03-14 16',
       '2005-03-14 17', '2005-03-14 18', '2005-03-14 19', '2005-03-14 20',
       '2005-03-14 21', '2005-03-14 22', '2005-03-14 23', '2005-03-15 00'],
      dtype='object')


In [4]:
bad_data_gen=list_1

In [5]:
for val in bad_data_gen:
    dx['temp_mean'].loc[val]= np.nan
    dx['rh_mean'].loc[val]= np.nan
    dx['atmos_pressure'].loc[val]= np.nan
    dx['wspd_arith_mean'].loc[val]= np.nan
    dx['wdir_vec_mean'].loc[val]= np.nan
    dx['pws_mean_vis_1min'].loc[val]= np.nan
    dx['pws_precip_rate_mean_1min'].loc[val]= np.nan
    dx['pws_cumul_snow'].loc[val]= np.nan

In [6]:
temp = dx['temp_mean']
temp_mask = temp.where(dx['qc_temp_mean'] ==0)
###############################################################################################
pressure = dx['atmos_pressure']
pressure_mask = pressure.where(dx['qc_atmos_pressure'] ==0)
###############################################################################################
rh = dx['rh_mean']
rh_mask = rh.where(dx['qc_rh_mean'] ==0)
###############################################################################################
wspd = dx['wspd_arith_mean']
wspd_mask = wspd.where(dx['qc_wspd_arith_mean'] ==0)
###############################################################################################
wdir = dx['wdir_vec_mean']
wdir_mask = wdir.where(dx['qc_wdir_vec_mean'] ==0)
###############################################################################################
vis = dx['pws_mean_vis_1min']
vis_mask = vis.where(dx['qc_pws_mean_vis_1min'] ==0)
###############################################################################################
precip_rate = dx['pws_precip_rate_mean_1min']
precip_rate_mask = precip_rate.where(dx['qc_pws_precip_rate_mean_1min'] ==0)
###############################################################################################
snow = dx['pws_cumul_snow']
snow_mask = snow.where(dx['qc_pws_cumul_snow'] ==0)

In [7]:
temp_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_temp_mask.nc')
pressure_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_pressure_mask.nc')
rh_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_rhmask.nc')
wspd_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_wspdmask.nc')
wdir_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_wdirmask.nc')
vis_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_vismask.nc')
precip_rate_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_precipratemask.nc')
snow_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2005_cumul_snow_mask.nc')